In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('lyrics.csv')
df.head()

,lyric,label
0,"Vou dar a volta no mundo eu vou, vou ver o mun...",axe
1,Nós duas Nós nuas E o nosso amor à toa Na boa ...,axe
2,Quem sabe nem era pra ser nossa Aquela lua aqu...,axe
3,Já são cinco da manhã E não dormi quase nada S...,axe
4,Amor de verdade eu só senti Foi com você meu b...,axe


In [3]:
def cat_to_num(label):
    if label == 'axe':
        label = 0
    elif label == 'sertanejo':
        label = 1
    elif label == 'gospelreligioso':
        label = 2
    elif label == 'bossa-nova':
        label = 3
    elif label == 'forro':
        label = 4
    elif label == 'mpb':
        label = 5
    elif label == 'samba':
        label = 6
    
    return label

In [4]:
df['label'] = df['label'].apply(lambda l: cat_to_num(l))
df.dtypes

lyric    object
label     int64
dtype: object

In [5]:
df.label.value_counts()

0    907
5    899
1    897
3    896
4    895
2    894
6    893
Name: label, dtype: int64

In [6]:
to_drop = []
for lyric, index in zip(df.lyric, df.index):
    if len(str(lyric)) <= 5:
        to_drop.append(index)
        
df.drop(to_drop, inplace=True)

In [7]:
from langdetect import detect

not_pt = []

for lyric, index in zip(df.lyric, df.index):
    #print(index)
    #print(detect(lyric))
    if len(lyric)<=0:
        not_pt.append(index)
    elif(detect(lyric) != 'pt'):
        not_pt.append(index)
        
print(len(not_pt))

df.drop(not_pt, inplace=True)

282


In [8]:
df.label.value_counts()

1    895
4    891
5    889
6    886
2    874
3    800
0    763
Name: label, dtype: int64

# Let's bora

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
sentences = df.lyric.values
y = df.label.values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

x_train = vectorizer.transform(sentences_train)
x_test = vectorizer.transform(sentences_test)

x_train[0]

<1x24899 sparse matrix of type '<class 'numpy.int64'>'
	with 49 stored elements in Compressed Sparse Row format>

# Machine Learning Approaches

In [17]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()

dt_model = dt_model.fit(x_train, y_train)

dt_score = dt_model.score(x_test, y_test)

print(dt_score)

0.374


In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=20, random_state=0)

rf_model = rf_model.fit(x_train, y_train)

rf_score = rf_model.score(x_test, y_test)

print(rf_score)

0.452


In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()

xgb_model = xgb_model.fit(x_train, y_train)

xgb_score = xgb_model.score(x_test, y_test)

print(xgb_score)

# Simple MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(1024,), random_state=1, max_iter=100)

#mlp_model.fit(x_train, y_train)                         

#mlp_score = mlp_model.score(x_test, y_test)

#print(mlp_score)